In [1]:
import pandas as pd
import numpy as np
import typing

In [2]:
def process_timestamp(**kwargs) -> pd.DataFrame:
    df = kwargs["df"]
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df["year"] = df["timestamp"].dt.year
    df["month"] = df["timestamp"].dt.month

    df.drop(["timestamp"], axis=1, inplace=True)

    return df

In [3]:
df = pd.read_csv('../Dataset/train.csv/train.csv')
# Create year/month of transactions
df = process_timestamp(df=df)
df = df[df.select_dtypes(include=np.number).columns.tolist() + ['sub_area']]
# df = df[['sub_area', 'year', 'month', 'price_doc']].groupby(by=["year", "month", 'sub_area']).median()
df['price_sqm'] = df['price_doc'] / df['full_sq']

In [4]:
grouped = df[['sub_area', 'year', 'price_sqm']].groupby(by=["year", 'sub_area']).median().unstack(level=0)
grouped

price_sqm                                               \
year                        2011           2012           2013           2014   
sub_area                                                                        
Ajeroport          163213.012478  178571.428571  174418.604651  198387.096774   
Akademicheskoe     150573.529412  169047.619048  186440.677966  200000.000000   
Alekseevskoe       184905.660377  169204.980843  181578.947368  193548.387097   
Altuf'evskoe                 NaN  148936.170213  142521.367521  143181.818182   
Arbat                        NaN  203875.573687  311379.754997  266472.749541   
...                          ...            ...            ...            ...   
Vyhino-Zhulebino   121334.185848  138450.168919  143462.171053  151389.961390   
Zamoskvorech'e     260869.565217  189851.097179  237010.777085  264705.882353   
Zapadnoe Degunino  125407.894737   88502.000000  100056.262427   91298.181818   
Zjablikovo                   NaN  154054.054054  154829.545455  155789.863115   
Zjuzino            152419.354839  161395.348837  172877.395577  172456.896552   

                                  
year                        2015  
sub_area                          
Ajeroport          205669.083718  
Akademicheskoe     203947.368421  
Alekseevskoe       216049.382716  
Altuf'evskoe       168892.580288  
Arbat              274444.444444  
...                          ...  
Vyhino-Zhulebino   149535.147392  
Zamoskvorech'e     256371.969249  
Zapadnoe Degunino   88494.000000  
Zjablikovo         166102.941176  
Zjuzino            175000.000000  

[146 rows x 5 columns]

In [5]:
tmp = grouped.to_dict()
new_dic = {}
for (_, year), vals in tmp.items():
    new_dic[year] = vals

grouped2 = pd.DataFrame.from_dict(new_dic)

In [6]:
mean_2015 = grouped2[2015].mean()
empty_sub_areas = grouped2[grouped2[2015].isnull()].index.tolist()
grouped2.loc[empty_sub_areas, 2015] = mean_2015

In [7]:
mean_2015/grouped2.loc[empty_sub_areas]

,2011,2012,2013,2014,2015
Poselenie Kievskij,NaN,NaN,NaN,1.286634,1.0
Poselenie Klenovskoe,NaN,NaN,7.066796,NaN,1.0
Poselenie Marushkinskoe,NaN,7.559828,1.878218,1.886741,1.0
Poselenie Mihajlovo-Jarcevskoe,NaN,NaN,NaN,2.266815,1.0
Poselenie Shhapovskoe,NaN,3.506007,NaN,3.804528,1.0
Vostochnoe,NaN,1.474883,1.289306,1.299730,1.0


In [9]:
grouped_copy = grouped2.copy()
for i in range(2011, 2016):
    grouped_copy[i] = grouped_copy[2015]/grouped_copy[i]

grouped_copy

,2011,2012,2013,2014,2015
Ajeroport,1.260127,1.151747,1.179169,1.036706,1.0
Akademicheskoe,1.354470,1.206449,1.093900,1.019737,1.0
Alekseevskoe,1.168430,1.276850,1.189837,1.116255,1.0
Altuf'evskoe,NaN,1.133993,1.185033,1.179567,1.0
Arbat,NaN,1.346137,0.881382,1.029916,1.0
...,...,...,...,...,...
Vyhino-Zhulebino,1.232424,1.080065,1.042332,0.987748,1.0
Zamoskvorech'e,0.982759,1.350384,1.081689,0.968516,1.0
Zapadnoe Degunino,0.705649,0.999910,0.884442,0.969285,1.0
Zjablikovo,NaN,1.078212,1.072812,1.066199,1.0


In [ ]:
grouped_copy.describe()

In [10]:
grouped_copy.to_csv('price_index.csv')

In [11]:
pi = pd.read_csv('price_index.csv', index_col=0)
pi_dic = pi.to_dict()
pi_dic

{'2011': {'Ajeroport': 1.2601267545744363,
  'Akademicheskoe': 1.354470265907958,
  'Alekseevskoe': 1.1684303350970018,
  "Altuf'evskoe": nan,
  'Arbat': nan,
  'Babushkinskoe': 1.238550721796995,
  'Basmannoe': 0.9429657571170036,
  'Begovoe': nan,
  'Beskudnikovskoe': 1.1158823529411763,
  'Bibirevo': 1.172031076581576,
  'Birjulevo Vostochnoe': 3.7,
  'Birjulevo Zapadnoe': 1.142918639690056,
  'Bogorodskoe': 1.3178939537797445,
  'Brateevo': 1.3537666074253885,
  'Butyrskoe': 1.2267369727047146,
  'Caricyno': 1.0965412808416342,
  'Cheremushki': 1.0979003288641538,
  "Chertanovo Central'noe": 1.6493725213234096,
  'Chertanovo Juzhnoe': 1.1894435351882162,
  'Chertanovo Severnoe': 1.3421052631578947,
  'Danilovskoe': 1.5824915824915826,
  'Dmitrovskoe': 1.2256977203635162,
  'Donskoe': 1.300533394196559,
  'Dorogomilovo': 1.335674986898308,
  'Filevskij Park': 1.3103448275862069,
  'Fili Davydkovo': 0.933766187596754,
  'Gagarinskoe': 1.189516129032258,
  "Gol'janovo": 1.276260504201

In [20]:
train_df = pd.read_csv("../Dataset/train.csv/train.csv")
train_df = process_timestamp(df=train_df)
i = 1
for year, vals in pi_dic.items():
    for sub_area, price_index in vals.items():
        train_df.loc[(train_df['sub_area'] == sub_area) & (train_df['year'] == int(year)), 'price_index'] = price_index


In [21]:
train_df[train_df['sub_area'] == 'Ajeroport']

,id,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,...,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc,year,month,price_index
156,157,44,29.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,22,51,0,16,118,14,6900000,2011,10,1.260127
262,263,102,66.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,12,30,0,5,100,14,17300000,2011,11,1.260127
298,299,43,28.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,19,41,0,7,110,15,8750000,2011,11,1.260127
544,545,54,37.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,25,53,0,23,125,14,8400000,2011,12,1.260127
598,599,63,35.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,48,89,1,54,146,12,12000000,2011,12,1.260127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27460,27463,151,NaN,15.0,1.0,1.0,2015.0,4.0,1.0,1.0,...,52,94,1,57,153,12,71249624,2015,1,1.000000
27833,27836,41,26.0,2.0,5.0,2.0,1961.0,2.0,5.0,3.0,...,12,31,0,5,98,14,8000000,2015,2,1.000000
28163,28166,37,22.0,9.0,9.0,5.0,1961.0,2.0,7.0,2.0,...,18,38,0,7,111,14,8000000,2015,3,1.000000
28560,28563,34,18.0,5.0,5.0,2.0,1960.0,1.0,8.0,2.0,...,24,53,0,15,119,14,6600000,2015,3,1.000000


In [22]:
train_df['price_index'].isnull().sum()

0